In [67]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
%matplotlib inline

import os
from google.colab import drive
drive.mount('/content/drive/',  force_remount=True)

import numpy as np

# Sort files in folder
from natsort import natsorted, ns

Mounted at /content/drive/


In [68]:
characters = ['0','1','2','3','4','5','6','7','8','9','-','+','times','(',')','y','=']
data_train_path = []
data_test_path =[]
for ch in characters:
  data_train_path.append(r"/content/drive/My Drive/Math_Solver/assets/pruned_dataset/" + ch + "/training/")
  data_test_path.append(r"/content/drive/My Drive/Math_Solver/assets/pruned_dataset/" + ch + "/testing/")

train_folder = []
for folder in data_train_path:
  train_folder.append([x for x in natsorted(os.listdir(folder))])
test_folder = []
for folder in data_test_path:
  test_folder.append([x for x in natsorted(os.listdir(folder))])

def readIMGFiles(Folder,p): # Folder, x and y dimnensions, p-path
  files = np.zeros((len(Folder), 32, 32), dtype=np.uint8)
  for i in range(len(Folder)):
    img = imread(p+Folder[i])[:,:]
    img = resize(img, (32, 32), mode='constant', preserve_range=True)
    files[i] = img.reshape(32,32)
  return files


## array reference key for paths
## 0 - 9 : numbers 0-9
## 10 : subtraction
## 11 : addition
## 12 : multiplication
## 13 : open parenthesis
## 14 : close parenthesis
## 15 : y-variable
## 16 : equal sign

In [80]:
training_data = readIMGFiles(train_folder[0],data_train_path[0])
testing_data = readIMGFiles(test_folder[0],data_test_path[0])
for i in range(1,3):
  training_data = np.concatenate((training_data,readIMGFiles(train_folder[i], data_train_path[i])))
  testing_data = np.concatenate((testing_data,readIMGFiles(test_folder[i], data_test_path[i])))
training_data = training_data[...,np.newaxis]
testing_data = testing_data[...,np.newaxis]
print(training_data.shape)

(2250, 32, 32, 1)


In [84]:
training_labels = np.zeros([750*3])
testing_labels = np.zeros([150*3])

j = 1
for i in range(750*3):
  if i >= j * 750:
    j += 1
  training_labels[i] = int(j - 1)
training_labels = training_labels[...,np.newaxis]

j = 1
for i in range(150*3):
  if i >= j * 150:
    j += 1
  testing_labels[i] = int(j - 1)
testing_labels = testing_labels[...,np.newaxis]

print(training_labels.shape)
print(training_labels[749])

print(testing_labels.shape)
print(testing_labels[100])

(2250, 1)
[0.]
(450, 1)
[0.]


In [78]:
# change as fit/needed

model = models.Sequential() # creates sequential model
# below are the network layers being added
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1))) # no rgb grey black n white
model.add(layers.MaxPooling2D((2, 2))) 
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))# 17 sybols/ charcters 

model.summary()

# change optimizer, loss, and metrics
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# change epochs as needed for higher accuracy
history = model.fit(training_data, training_labels, epochs=10, batch_size=1)


Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_46 (Conv2D)           (None, 30, 30, 32)        320       
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten_15 (Flatten)         (None, 1024)              0         
_________________________________________________________________
dense_30 (Dense)             (None, 64)              

In [99]:
prediction = model.predict(testing_data, verbose=1)
print(prediction[306])
testing_data

15/15 [==============================] - 0s 10ms/step
[4.2302334e-05 4.1878852e-03 9.9576980e-01]


In [ ]:
!pip install jupyter-dash
!pip install dash-canvas==0.1.0
!pip install dash==1.17.0

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
from dash_canvas import DashCanvas
import plotly.express as px
from jupyter_dash import JupyterDash


In [ ]:
# Load Data
# df = px.data.tips()
# filename = 'https://drive.google.com/file/d/1hYD2JiDmR6qLFwUbb7pO2d8KEg0ToMU8/view?usp=sharing'
# contains javascript function to install image
# external_script = 'https://drive.google.com/file/d/1KzrU7av15opjPjZnPomXFQtPdMJb960j/view?usp=sharing'
# external_css = 'https://drive.google.com/file/d/1-16FYwgI4_7Ocyg7Fmjs_eJQQJbqtB7J/view?usp=sharing'
# Build App
# app = JupyterDash(__name__, assets_external_path=external_css, external_stylesheets=external_css)
app = JupyterDash(__name__)
app.layout = html.Div(html.Center([
    html.H1("My Math Solver",style={"color":"cornflowerblue", "textAlign":"center"}),
    html.Div(
      DashCanvas(id='canvas', 
                 hide_buttons=['zoom', 'line', 'select', 'rectangle'],
                 lineColor='black',
                 lineWidth=5,
                 width=400),
                 style={"margin":"auto","border":"2px solid black", "width": "400px"}
    ),
    html.Div(
        dcc.Input(
            id='text_field', 
            type='text',
            placeholder="Enter equation here!")
        ),
    html.Button('Submit', id='submit_button', n_clicks=0),
    html.Div(id='container-button-basic',
             children='Enter a equation and press submit'),
    html.Img(id='my-image', width=300),
    html.P(id='test-p', className='test-p', children='Some test string to see if css is working')
  ]))

@app.callback(Output('my-image', 'src'),[Input('canvas', 'json_data')])
def update_data(string):
    if string:
        mask = parse_jsonstring(string, io.imread(filename, as_gray=True).shape)
    else:
        raise PreventUpdate
    return array_to_data_url(img_as_ubyte(src))

# app.run_server(mode='external')
# Run app and display result inline in the notebook
# app.css.append_css(dict(external_url=external_css))
# app.run_server(mode='external', host='127.0.0.1', port=8050, debug=True, use_reloader=False)

In [ ]:
app._terminate_server_for_port('127.0.0.1', 8050)